In [2]:
import deepchem as dc
deepchem.__version__

'2.7.1'

# Splitters
En DeepChem, **un método para dividir muestras en múltiples conjuntos de datos** se define mediante un **objeto Splitter.** Elegir un método adecuado para tus datos es muy importante. De lo contrario, tu **modelo entrenado podría parecer funcionar mucho mejor** de lo que **realmente lo hace.**

Considera un típico proceso de desarrollo de medicamentos. Es posible que comiences evaluando miles de moléculas para ver si se unen a tu objetivo de interés. Una vez que encuentras una que parece funcionar, intentas optimizarla probando miles de variaciones menores en ella, buscando una que se una de manera más fuerte. Luego, quizás la pruebas en animales y descubres que tiene una toxicidad inaceptable, por lo que pruebas más variaciones para solucionar los problemas.

Esto tiene una consecuencia importante para **los conjuntos de datos químicos: a menudo incluyen muchas moléculas que son muy similares entre sí. Si divides los datos en conjuntos de entrenamiento y prueba de una manera ingenua, el conjunto de entrenamiento incluirá muchas moléculas que son muy similares a las que están en el conjunto de prueba, incluso si no son exactamente idénticas. Como resultado, el modelo puede funcionar muy bien en el conjunto de prueba, pero luego fallar estrepitosamente cuando intentas usarlo en otros datos que son menos similares a los datos de entrenamiento.**

Echemos un vistazo a algunos de los splitters que se encuentran en DeepChem.

## RandomSplitter
Este es uno de los splitters más simples. Simplemente selecciona muestras para los conjuntos de **entrenamiento, validación y prueba de una manera completamente aleatoria.**

¿No dijimos que esto es una mala idea? Bueno, depende de tus datos. **Si cada muestra es verdaderamente independiente de las demás, entonces esta es una forma tan buena como cualquier otra de dividir los datos.** No hay una elección universalmente mejor de splitter. Todo depende de tu conjunto de datos en particular, y para algunos conjuntos de datos, esta es una elección adecuada.

## RandomStratifiedSplitter
**Algunos conjuntos de datos son muy desequilibrados:** solo una fracción muy pequeña de todas las muestras son positivas. En ese caso, la división aleatoria a veces puede llevar a que el conjunto de validación o prueba tenga pocas o incluso ninguna muestra positiva para algunas tareas. Eso hace que sea imposible evaluar el rendimiento.

**RandomStratifiedSplitter aborda esto dividiendo equitativamente las muestras positivas y negativas.** Si solicitas una división del 80/10/10, los conjuntos de validación y prueba no solo contendrán un 10% de muestras, sino también un 10% de las muestras positivas para cada tarea.

## ScaffoldSplitter
Este splitter intenta abordar el problema discutido anteriormente, donde **muchas moléculas son muy similares entre sí.** **Identifica la estructura base (scaffold) que forma el núcleo de cada molécula y asegura que todas las moléculas con el mismo scaffold se coloquen en el mismo conjunto de datos.** Aún no es una solución perfecta, ya que dos moléculas pueden tener scaffolds diferentes pero ser muy similares en otras formas, pero generalmente es una gran mejora en comparación con la división aleatoria.

## ButinaSplitter
Este es otro splitter que intenta abordar el **problema de las moléculas similares.** **Las agrupa en función de sus huellas moleculares, de modo que las que tienen huellas similares tienden a estar en el mismo conjunto de datos.** El tiempo requerido por este algoritmo de división **escala al cuadrado del número de moléculas,** por lo que es principalmente útil para **conjuntos de datos pequeños a medianos.**

## SpecifiedSplitter
**Este splitter deja todo en manos del usuario. Le indicas exactamente qué muestras poner en cada conjunto de datos.** Esto es útil cuando **sabes de antemano que una división particular es apropiada para tus datos.**

Un ejemplo es la división temporal. Considera un proyecto de investigación en el que estás generando y probando constantemente nuevas moléculas. A medida que obtienes más datos, vuelves a entrenar periódicamente tu modelo en el conjunto de datos en constante crecimiento y lo usas para predecir resultados para otras moléculas que aún no se han probado. Una buena forma de validar si esto funciona es elegir una fecha de corte específica, entrenar el modelo con todos los datos que tenías en ese momento y ver qué tan bien predice otros datos que se generaron posteriormente.


## Efectos de usar diferentes Splitters

Probemos a usar ahora tres tipos de splitters para un mismo conjunto de datos: random, scaffold y butina.
Entrenaremos y evaluaremos un modelo usando esos splitters

In [3]:
splitters = ['random', 'scaffold', 'butina']
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for splitter in splitters:
    tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', split=splitter)
    train_dataset, valid_dataset, test_dataset = datasets
    model = dc.models.MultitaskClassifier(n_tasks=len(tasks), n_features=1024, layer_sizes=[1000])
    model.fit(train_dataset, nb_epoch=10)
    print('splitter:', splitter)
    print('training set score:', model.evaluate(train_dataset, [metric], transformers))
    print('test set score:', model.evaluate(test_dataset, [metric], transformers))
    print()

'split' is deprecated.  Use 'splitter' instead.
[09:50:02] WARNING: not removing hydrogen atom without neighbors


splitter: random


'split' is deprecated.  Use 'splitter' instead.


training set score: {'roc_auc_score': 0.9543137257860703}
test set score: {'roc_auc_score': 0.7678270060449034}

splitter: scaffold


'split' is deprecated.  Use 'splitter' instead.


training set score: {'roc_auc_score': 0.957978096915077}
test set score: {'roc_auc_score': 0.6817174026529185}



[09:50:55] WARNING: not removing hydrogen atom without neighbors
[09:51:06] WARNING: not removing hydrogen atom without neighbors


splitter: butina
training set score: {'roc_auc_score': 0.9580935374570024}
test set score: {'roc_auc_score': 0.6052172942948477}



Podemos ver que todos ellos producen un rendimiento similar en el entrenamiento pero en el random splitter tiene muchísimo más rendimiento el test set. 

¿Significa eso que el random splitting es mejor? Realmente no. Sifnifica que random splitting no te da una meduda precisa de como tu modelo funciona realmente. Esto es debido a que el test set contiene moléculas que son muy similares en el training set, lo que hace que realmente no sea independiente. **Lo que provoca es que el modelo parezca que funciona mejor que lo que lo hace. Scaffold y Butina splitting te dan una mejor indicación de lo que puedes esperar con datos independientes en el futuro**